In [2]:
import time, random

In [3]:
import numpy as np
import itertools
from py2opt.routefinder import RouteFinder
from scipy.spatial import distance_matrix
def tsp_opt(points):
    """
    Dynamic programing solution for TSP - O(2^n*n^2)
    https://gist.github.com/mlalevic/6222750

    :param points: List of (x, y) points
    :return: Optimal solution
    """

    def length(x_coord, y_coord):
        return np.linalg.norm(np.asarray(x_coord) - np.asarray(y_coord))

    # Calculate all lengths
    all_distances = [[length(x, y) for y in points] for x in points]
    # Initial value - just distance from 0 to every other point + keep the track of edges
    A = {(frozenset([0, idx+1]), idx+1): (dist, [0, idx+1]) for idx, dist in enumerate(all_distances[0][1:])}
    cnt = len(points)
    for m in range(2, cnt):
        B = {}
        for S in [frozenset(C) | {0} for C in itertools.combinations(range(1, cnt), m)]:
            for j in S - {0}:
                # This will use 0th index of tuple for ordering, the same as if key=itemgetter(0) used
                B[(S, j)] = min([(A[(S-{j}, k)][0] + all_distances[k][j], A[(S-{j}, k)][1] + [j])
                                 for k in S if k != 0 and k != j])
        A = B
    res = min([(A[d][0] + all_distances[0][d[1]], A[d][1]) for d in iter(A)])
    return np.asarray(res[1])

def tsp_2_opt(points):
    """
    2-Opt programing solution for TSP
    https://github.com/pdrm83/py2opt

    :param points: List of (x, y) points
    :return: Best solution
    """
    point_idx = list(range(len(points)))
    dist_mat = distance_matrix(points, points)

    route_finder = RouteFinder(dist_mat, point_idx, iterations=5)
    best_distance, best_route = route_finder.solve()

    return np.array(best_route), best_distance

In [4]:
import numpy as np

#Reading the data from text files 
#Real world datasets or collected through a process not random, in form of coordinates
#Some datasets are with different number of cities , the biggest dataset with consistent number of cities and optimally ordered instances is TSP50 and TSP100 datasets split up into training and test datasets.
#The datasets have already an optimal solution because they are ordered in the dataset , when we use the dataset we have to shuffle the various instances
#Heuristic methods can also be used to compare to the given optimal solutions
#Dynamic programming method takes too much time but other heuristic methods and deep learning methods tackle the complexity of the dataset , very easy to use since the given dataset is (50, 2) or (100, 2) as an instance in form of coordinates and input becomes easy to manage, output is a sequence of optimally ordered cities in size (50, 1) and optimal cost as a floating number which uses Euclidean distance



def read_coordinates_from_file(file_path):
    coordinates = []
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:
                try:
                    x = float(parts[0])
                    y = float(parts[1])
                except(ValueError):
                    continue
                coordinates.append((x, y))
    return coordinates

file_path_test_50 = r"tsp_unif_test50_100_100000.txt"
file_path_train_50 = r"tsp_unif50_10000_100000_1000.txt"
file_path_test_100 = r"tsp_unif_test100_100_100000.txt"
file_path_train_100 = r"tsp_unif100_10000_100000_1000.txt"

coordinates_test50 = read_coordinates_from_file(file_path_test_50)
test50 = np.array(coordinates_test50)
coordinates_train50 = read_coordinates_from_file(file_path_train_50)
train50 = np.array(coordinates_train50)

coordinates_test100 = read_coordinates_from_file(file_path_test_100)
test100 = np.array(coordinates_test100)

coordinates_train100 = read_coordinates_from_file(file_path_train_100)
train100 = np.array(coordinates_train100)

print(test50.shape, train50.shape, test100.shape, train100.shape)

testinstances50 = []
for i in range(0, 5000, 50):
    testinstances50.append(test50[i:i+50, :])
testinstances50 = np.array(testinstances50)
optimal_distances_test50 = []
for i in range(testinstances50.shape[0]):
    distance = 0
    for j in range(testinstances50.shape[1]-1):
        distance += np.linalg.norm(testinstances50[i][j] -  testinstances50[i][j+1])
    optimal_distances_test50.append(distance)
optimal_distances_test50 = np.array(optimal_distances_test50)

traininstances50 = []
for i in range(0, 1500000, 50):
    traininstances50.append(train50[i:i+50, :])
traininstances50 = np.array(traininstances50)
optimal_distances_train50 = []
for i in range(traininstances50.shape[0]):
    distance = 0
    for j in range(traininstances50.shape[1]-1):
        distance += np.linalg.norm(traininstances50[i][j] -  traininstances50[i][j+1])
    optimal_distances_train50.append(distance)
optimal_distances_train50 = np.array(optimal_distances_train50)

testinstances100 = []
for i in range(0, 10000, 100):
    testinstances100.append(test100[i:i+100, :])
testinstances100 = np.array(testinstances100)
optimal_distances_test100 = []
for i in range(testinstances100.shape[0]):
    distance = 0
    for j in range(testinstances100.shape[1]-1):
        distance += np.linalg.norm(testinstances100[i][j] -  testinstances100[i][j+1])
    optimal_distances_test100.append(distance)
optimal_distances_test100 = np.array(optimal_distances_test100)

traininstances100 = []
for i in range(0, 1000000, 100):
    traininstances100.append(train100[i:i+100, :])
traininstances100 = np.array(traininstances100)
optimal_distances_train100 = []
for i in range(traininstances100.shape[0]):
    distance = 0
    for j in range(traininstances100.shape[1]-1):
        distance += np.linalg.norm(traininstances100[i][j] -  traininstances100[i][j+1])
    optimal_distances_train100.append(distance)
optimal_distances_train100 = np.array(optimal_distances_train100)

(5000, 2) (1500000, 2) (10000, 2) (1000000, 2)


In [5]:
mean_test100 = np.mean(optimal_distances_test100)
mean_test50 = np.mean(optimal_distances_test50)
mean_train100 = np.mean(optimal_distances_train100)
mean_train50 = np.mean(optimal_distances_train50)

In [5]:
# 2 opt method on traininstances100
random.seed(1)
start = time.time()
y_pred = []
for i in traininstances100:
    np.random.shuffle(i)
    route, yp = tsp_2_opt(i)
    y_pred.append(yp)
end = time.time()
print(end-start)
abs_error = abs(mean_test50 - np.mean(y_pred))
relative_error = abs_error/mean_test50
print(abs_error, relative_error)

0 msec
82 msec
142 msec
191 msec
250 msec
0 msec
58 msec
107 msec
174 msec
226 msec
0 msec
50 msec
99 msec
150 msec
213 msec
0 msec
50 msec
101 msec
161 msec
228 msec
0 msec
57 msec
107 msec
168 msec
227 msec
0 msec
40 msec
81 msec
149 msec
207 msec
0 msec
62 msec
113 msec
165 msec
225 msec
0 msec
71 msec
123 msec
184 msec
237 msec
0 msec
61 msec
112 msec
172 msec
225 msec
0 msec
61 msec
121 msec
161 msec
212 msec
0 msec
65 msec
130 msec
189 msec
241 msec
0 msec
67 msec
117 msec
165 msec
232 msec
0 msec
56 msec
117 msec
179 msec
249 msec
0 msec
72 msec
126 msec
197 msec
260 msec
0 msec
59 msec
128 msec
195 msec
301 msec
0 msec
50 msec
110 msec
161 msec
220 msec
0 msec
67 msec
117 msec
175 msec
223 msec
0 msec
56 msec
120 msec
191 msec
243 msec
0 msec
52 msec
114 msec
175 msec
228 msec
0 msec
59 msec
126 msec
176 msec
228 msec
0 msec
55 msec
119 msec
169 msec
226 msec
0 msec
60 msec
111 msec
163 msec
230 msec
0 msec
59 msec
120 msec
191 msec
253 msec
0 msec
59 msec
109 msec
166 msec
224

Traceback (most recent call last):
  File "/apps/jupyterhub/jh3.1.1-py3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3703456/2712902058.py", line 7, in <module>
    route, yp = tsp_2_opt(i)
                ^^^^^^^^^^^^
  File "/tmp/ipykernel_3703456/1055062608.py", line 45, in tsp_2_opt
    best_distance, best_route = route_finder.solve()
                                ^^^^^^^^^^^^^^^^^^^^
  File "/home/hpc/iwso/iwso104h/.local/lib/python3.11/site-packages/py2opt/routefinder.py", line 32, in solve
    new_route, new_distance, distances = tsp.two_opt()
                                         ^^^^^^^^^^^^^
  File "/home/hpc/iwso/iwso104h/.local/lib/python3.11/site-packages/py2opt/solver.py", line -1, in two_opt
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/apps/jupyterhub/jh3.1.1-p

In [ ]:
#City swap method on traininstances100

import numpy as np
import math as math
import pandas as pd

def objective_calculator(solution,dataset): #Calculates the objective function value (cost) of any solution (tour)
    cost = 0
    for i in range(len(solution)-2):
        cost += euclid_calculator(solution[i], solution[i+1],dataset)  #To the euclid_calculator function, send the cities in the solution whose objective function value will be calculated, two at a time

    return cost


def euclid_calculator(city_1, city_2, dataset): #Calculates the euclidean distance between any two cities in the dataset

    return math.sqrt((dataset.loc[city_1-1,"x"]-dataset.loc[city_2-1,"x"])**2 + (dataset.loc[city_1-1,"y"]-dataset.loc[city_2-1,"y"])**2) #Calculates the euclidean distance with formula using the city coordinates in the "dataset" dataframe



def city_swap(city_1,city_2,current_solution,dataset):

    tour_choice=current_solution.copy()                #In these lines, an array called tour_choice is created to try the swap on that array first.
    keeper=tour_choice[city_1].copy()
    tour_choice[city_1]=tour_choice[city_2].copy()
    tour_choice[city_2]=keeper

    if objective_calculator(tour_choice,dataset) < objective_calculator(current_solution,dataset): #The objective function values ​​of the new tour we tried and the previous tour are compared, checking if the new solution is better      #The objective function value of the tour, which is the better solution, is printed
        current_solution=tour_choice
    return current_solution


def main(dataset): #Argument of the main function is the dataset of the coordinates of the cities in "euclidean space"
    np.random.seed(28) #You can choose random seed
    partly_initial_solution= np.random.permutation(range(1,len(dataset)+1))  #Randomly sorts the city numbers and creates the starting tour (starting solution)
    initial_solution = np.append(partly_initial_solution, [partly_initial_solution[0]]) #Adds the city at the beginning of the tour to the end of the tour to make the salesman return to where he started

    current_solution = initial_solution #Assigns the initial solution to the current solution
    for k in range(10): #Trying to swap all cities with each other using nested for loops (You can change that "10" as you wish)
        for i in range(1,len(dataset)-1):
            for j in range(i+1,len(dataset)):
                current_solution = city_swap(i,j,current_solution,dataset) #The cities to be swapped in the loop are sent to the city_swap function, with the current solution and dataset

    return objective_calculator(current_solution,dataset)

def city_swap1(instance):
    np.random.shuffle(instance)
    new_instance = pd.DataFrame(instance)
    new_instance = new_instance.rename(columns = {0:"x", 1: "y"})
    sol = main(new_instance)
    return sol

start = time.time()
ypred = []
c=0
for i in traininstances100:
    result = city_swap1(i)
    ypred.append(result)
end = time.time()
print(end - start)
abs_error = abs(mean_train100 - np.mean(ypred))
relative_error = abs_error/mean_train100
print(abs_error, relative_error)

In [ ]:
# Genetic algorithm on traininstances100

import numpy as np
import pandas as pd
import time                    #Required library to calculate Computational Time
import random

def calculate_distance(cities , solution):
    solution_for_distance_calculation = np.append(solution, [solution[0]], axis=0) #Appends the city index at the beginning of the solution array to the end of the array
    distance = 0
    next_city_index_founder=0

    for i in solution_for_distance_calculation: #i will hold first city indexes
        next_city_index_founder += 1
        if next_city_index_founder < len(solution_for_distance_calculation):
            next_city_index=solution_for_distance_calculation[next_city_index_founder] #Find the second city indexes here
            distance += np.sqrt(((cities[next_city_index,0]-cities[i,0])**2)+((cities[next_city_index,1]-cities[i,1])**2)) #First city and second city indexes are used when calculating euclidean distance

    return distance

def parent_selection(population, number_of_pairs_M):
    current_parents = []

    #Parent selection from a population
    parent_counter = 1

    while parent_counter <= 2*number_of_pairs_M: #We will select twice as many parents as the desired number of parent "pairs" i.e. M, so a parent will be selected every time this loop is iterated

        random_float = random.uniform(0,population["fitness"].sum()) #A float number is randomly selected in the range of 0 and the sum of fitness values
        cumulative_counter = 0 #Variable to assign the larger number in the cumulative test

        for solution, fitness in population.itertuples(index=False):

            cumulative_counter_copy = cumulative_counter   #cumulative_counter_copy is the variable to assign the smaller number in the cumulative test
            cumulative_counter += fitness

            if cumulative_counter_copy <= random_float <= cumulative_counter:   #If the randomly generated float number is in the cumulative range, the parent in question is selected

                append_checker = True #But first, check if the solution in question is already in the current parent list
                for parent in current_parents:
                    if parent is solution:
                        append_checker = False


                if append_checker == True: #If the solution in question is not found in the current parent list, it is appended
                    current_parents.append(solution)
                    parent_counter += 1

    return current_parents


def crossover(current_parents, crossover_probability):
    children = [] #Children created with crossover will be kept in this list
    for parent_index_holder in range(1, len(current_parents)): #Loop created to iterate from the second parent
        if random.uniform(0,1) < crossover_probability: #Crossover to parent pairs with the probability specified in the crossover_probability argument

            parent_1 = current_parents[parent_index_holder-1]
            parent_2 = current_parents[parent_index_holder]

            left_bound = random.randint(1, len(current_parents[0])) #left border of crossover is randomly determined
            right_bound = random.randint(left_bound, len(current_parents[0])) #right border of crossover is randomly determined

            #Child creation as a result of crossover is done here
            child =np.array([]) #An empty array is created to create its child
            for j in range(left_bound): #The part of the child from the beginning to the left bound comes from parent 1
                child = np.append(child, parent_1[j])

            for k in range(left_bound,right_bound): #The part of child between left bound and right bound comes from parent 2
                child = np.append(child, parent_2[k])

            for l in range(right_bound, len(parent_1)): #The part of the child from the right bound to the end comes from parent 1
                child = np.append(child, parent_1[l])

            #Mappings for currently created children are created here
            maps_list = []
            for m in range(left_bound, right_bound):
                maps_list.append([parent_1[m],parent_2[m]])

            #Fix the infeasible child here
            child = infeasible_child_fixer(child, maps_list)

            #Created child are appended to the children array
            children.append(child)

    return children

def infeasible_child_fixer(child, maps_list):
    #print("Mappings: ",maps_list)              #You can print mappings for current child if you want
    #print("Ve child ilk hali bu: " , child)    #You can print current child before fixing

    i=1
    while i==1:

        controlled_city_index_holder = -1
        for controlled_city in child: #The number of each city in child will be checked
            controlled_city_index_holder += 1

            city_counter = 0 #This variable will keep the number of currently checked city in that child solution
            for city in child: #The number of currently checked city is found in this for loop
                if city == controlled_city:
                    city_counter += 1

            if city_counter < 2:

                will_break = False

            if city_counter > 1: #If controlled city is more than 1 in the current child solution; we need to replace controlled_city with the city it is mapped to
                for a_map in maps_list:
                    if a_map[0] == controlled_city: #Mapping where controlled_city is located

                        child[controlled_city_index_holder] = a_map[1] #Replace the controlled city in the child solution with the other city in that mapping

                        will_break = True

                        maps_list.remove(a_map)          #Used mapping is removed from the mapping list

                        break

                    elif a_map[1] == controlled_city: #Mapping where controlled_city is located

                        child[controlled_city_index_holder] = a_map[0] #Replace the controlled city in the child solution with the other city in that mapping

                        will_break = True

                        maps_list.remove(a_map)         #Used mapping is removed from the mapping list

                        break


            if will_break:
                break
        #print("This is the new version of the child solution after the change: ",child)        #You can print the new version of the child solution after the change

        #There was a change in the child solution, so we have to start the checking process from the beginning

        #But first we check if the child solution is fixed
        child_fixed = True
        city_counts = []
        for city in child:
            count = 0

            for check in child:
                if city == check:
                    count += 1

            city_counts.append([city, count])
        #print("Here is the list of cities in the new version of the child solution and how many they are:", city_counts)   #You can print the list of cities in the new version of the child solution and how many they are

        #Check if any city is more than 1 in the new child solution
        for count in city_counts:
            if count[1] > 1:
                child_fixed = False

        #If the child solution is fixed, we finish checking it.
        if child_fixed:
            i=2
            break

    #print("Fixed version of that child solution: ", child) #You can print the fixed version of that child solution
    return child


#Apply mutation to child solutions, with a probability, by inverting a random part of it
def mutate_children(children, mutation_probability):
    children_after_mutation = []

    for child in children:
        if random.uniform(0, 1) <= mutation_probability:
            left_bound = random.randint(0,len(child))
            right_bound = random.randint(left_bound,len(child))
            child[left_bound:right_bound] = child[left_bound:right_bound][::-1]
            children_after_mutation.append(child)
        else:
            children_after_mutation.append(child)

    return children_after_mutation

def generation_creator(population, mutated_children, cities):
    #A dataframe named "children" containing children and fitness values is created
    integer_mutated_children = []
    mutated_children_fitnesses = []
    for child in mutated_children:
        child = child.astype(int)
        integer_mutated_children.append(child)
        distance = calculate_distance(cities,child)
        fitness = 1/distance
        mutated_children_fitnesses.append(fitness)
    children = pd.DataFrame(list(zip(integer_mutated_children,mutated_children_fitnesses)),columns=['solution','fitness'])
    children.sort_values(by='fitness',axis=0,inplace=True,ascending=False)

    #The best half of the children are selected to be included in the population
    choosen_children_number = round(len(children)/2)
    choosen_children = children.head(choosen_children_number)

    #From the worst members of the current population, as many solutions as children to be added are discarded
    population = population.head(len(population)-choosen_children_number)

    #Selected children are added to the remaining solutions in the population; new population is also sorted by fitness
    new_population = pd.concat([population, choosen_children])
    new_population.sort_values(by='fitness',axis=0,inplace=True,ascending=False)

    return new_population





def main(generation_number, number_of_individuals, number_of_pairs_M, crossover_probability, mutation_probability):

    #k = 0 #keeps the current generation number

    #A dataframe named "population" containing initial population and fitness values is created
    #solutions = []
    #fitnesses = []
    for j in traininstances100:
        solutions = []
        fitnesses = []
        k = 0
        for i in range(0,number_of_individuals): #for loop's range is number_of_individuals, since there will be as many individuals in the population as are entered as argument
            solution=np.random.permutation(len(np.array(j)))
            solutions.append(solution)
            distance = calculate_distance(np.array(j), solution)
            fitness = 1/distance                 #The fitness value of a solution (i.e. an individual) is calculated with 1/distance
            fitnesses.append(fitness)
        population = pd.DataFrame(list(zip(solutions,fitnesses)),columns=['solution','fitness'])
        population.sort_values(by='fitness',axis=0,inplace=True,ascending=False)  #Individuals in the population are ranked in descending order of fitness values

        print("Initial population: ")  #Initial population is printed
        print(population)

        #Genetic search starts (new generations will be produced as many as the desired generation number)
        for i in range(generation_number):
            k+=1

        #parents are created to produce the next generation
            current_parents = parent_selection(population, number_of_pairs_M)


        #Child solutions are created by crossover
            children = crossover(current_parents, crossover_probability)


        #Child solutions are mutated with a probability
            mutated_children = mutate_children(children, mutation_probability) #inversion mutation uyguladık


        #Replacement is done and new generation is created
            population = generation_creator(population, mutated_children, np.array(j))
            print("--------------------------")
            print("Generation number: ",k )
            
            
            if k == generation_number:
                print(population)
                for solution, fitness in population.itertuples(index=False):
                    print("Best solution founded: ", np.append(solution, [solution[0]], axis=0))
                    print("Cost of that solution: ", calculate_distance(np.array(j) , solution) )

            break


start_time = time.time()                 #Keeps the start time

main(1500, 100, 25, 0.7, 0.5) #(generation number, number of individuals in a generation, Number of parent "pairs" to be selected in parent selection, crossover probability for a parent pair, mutation probability for a child solution)

comp_time = time.time() - start_time     #Subtracts the start time from the end time and keeps the result
print(f"-> Computational Time: {comp_time} seconds")     #Prints computational time

Initial population: 
                                             solution       fitness
95  [52, 98, 26, 28, 57, 59, 54, 37, 30, 24, 92, 3...  2.008085e-07
91  [78, 55, 18, 89, 17, 5, 44, 22, 37, 45, 34, 66...  1.981157e-07
1   [1, 34, 99, 91, 20, 56, 45, 70, 35, 0, 97, 86,...  1.971480e-07
42  [83, 29, 26, 96, 93, 23, 85, 87, 5, 9, 40, 82,...  1.970080e-07
27  [51, 37, 72, 65, 56, 47, 36, 50, 88, 97, 67, 3...  1.961101e-07
..                                                ...           ...
3   [58, 32, 49, 82, 19, 22, 34, 57, 96, 7, 4, 94,...  1.695382e-07
45  [77, 24, 54, 29, 22, 30, 33, 12, 0, 69, 49, 38...  1.683327e-07
92  [17, 0, 35, 60, 87, 97, 15, 79, 48, 1, 36, 78,...  1.680846e-07
19  [90, 60, 78, 38, 81, 25, 97, 34, 21, 55, 68, 1...  1.676584e-07
85  [69, 55, 18, 62, 92, 35, 23, 70, 8, 51, 99, 85...  1.663769e-07

[100 rows x 2 columns]
--------------------------
Generation number:  1
                                             solution       fitness
95  [52, 98, 26, 28, 5

In [7]:
#Simulated annealing on traininstances100

import numpy as np
import pandas as pd
import time                    #Required library to calculate Computational Time

start_time = time.time()                 #Keeps the start time

np.random.seed(40)

def calculate_distance(cities , solution):
    solution_for_distance_calculation = np.append(solution, [solution[0]], axis=0) #Appends the city index at the beginning of the solution array to the end of the array
    distance = 0
    next_city_index_founder=0

    for i in solution_for_distance_calculation: #i will hold first city indexes
        next_city_index_founder += 1
        if next_city_index_founder < len(solution_for_distance_calculation):
            next_city_index=solution_for_distance_calculation[next_city_index_founder] #Find the second city indexes here
            distance += np.sqrt(((cities[next_city_index,0]-cities[i,0])**2)+((cities[next_city_index,1]-cities[i,1])**2)) #First city and second city indexes are used when calculating euclidean distance

    return distance

def generate_solution(current_solution): #A new solution will be created by swapping two random cities in the current solution
    idx1 , idx2 = np.random.choice(len(current_solution),2)
    current_solution_copy = current_solution.copy()
    current_solution_copy[idx2], current_solution_copy[idx1] = current_solution_copy[idx1], current_solution_copy[idx2]
    return current_solution_copy



def main(dataset, T, cooling_rate, T_lower_bound, tolerance):
    
    for i in dataset:
        data = pd.DataFrame(i)
        cities = np.array(i)

        current_solution = np.random.permutation(range(len(data))) #A random initial solution is created using city indexes
        h=0 #Keeps the number of iterations
        T_new = T
        while T_new > T_lower_bound: #We want the algorithm to run when the temperature is greater than T_lower_bound
            h+=1
            while True: #Local search will be done here; different solutions will be tried for the "same" temperature value, until new solutions give very close values ​​to the current solution (that is, until the difference in costs between the new potential solution and the current solution is less than the tolerance)
                potential_solution = generate_solution(current_solution) #The potential solution is created using the generate_solution function
                potential_distance = calculate_distance(cities , potential_solution) #The cost of the potential solution is calculated with the calculate_distance function
                current_distance = calculate_distance(cities , current_solution) #The cost of the current solution is calculated with the calculate_distance function


                if potential_distance < current_distance: #If the potential solution is better, the potential solution is accepted
                    current_solution = potential_solution


                elif np.random.random() < np.exp(-(potential_distance - current_distance)/T): #Potential solution has a chance to be accepted based on a probability even if it is worse
                    current_solution = potential_solution


                if np.abs(potential_distance-current_distance) < tolerance: #Local search will run until a potential solution gives a cost value very close to the current solution
                    break


            T_new = T_new*cooling_rate #The temperature is updated depending on the cooling rate

        print("--------RESULTS---------")
        print("Best tour founded for salesman: ", np.append(current_solution, [current_solution[0]], axis=0))
        print("Distance of tour founded: ", current_distance)
        print("Iterations: ",h )
    comp_time = time.time() - start_time     #Keeps the difference between the end time and the start time
    print(f"-> Computational Time: {comp_time} seconds")     #Prints Computational Time

main(traininstances100, 100 , 0.999, 0.01, 1)   #(Dataset including city coordinates, Starting temperature, Cooling rate, Lower bound of temperature, Tolerance of local search) You can set these arguments as you want

--------RESULTS---------
Best tour founded for salesman:  [40 39 38 37 36 31 30 28 27 29 34 43 45 46 44 21 17 16  4  3  1 99 98 97
 94 93 91 88 19 18 20 26 25 79 77 76 78 80 82 83 13 14 15 12 11 75 74 73
 72 92 90  2  5  7 89 84 22 24 23 32 33 35 42 48 47 50 49 62 64 95 96  0
  6  8  9 10 87 86 85 81 54 55 56 57 58 60 61 69 71 70 68 67 66 65 63 59
 51 52 53 41 40]
Distance of tour founded:  1440961.6054873746
Iterations:  9206
--------RESULTS---------
Best tour founded for salesman:  [97 98 94 93 92 77 76 72 91 84 32 33 30 29 39 40 41 43 42 79 80 36 37 54
 74 75 78 81 83 82 44 46 47 51  0  1  2  6 89 88 35 38 53 52 55 56 57 58
 59 60 61 62 63 64 90  7  4  3  5 67 66 65 73 28 27 26 25 24 23 31 34 50
 49 48 45 85 86 87 13 15 16 12  8  9 10 11 21 22 20 19 18 17 14 99 95 96
 68 69 70 71 97]
Distance of tour founded:  1666864.0128779013
Iterations:  9206


KeyboardInterrupt: 

In [9]:
optimal_distances_train100[1]

769698.0359091841